In [1]:
!pip install gpustat
!pip install --upgrade keras
!pip install --upgrade keras_preprocessing
!pip install segmentation_models
!gpustat

     |████████████████████████████████| 81kB 4.7MB/s 
  Created wheel for gpustat: filename=gpustat-0.6.0-cp36-none-any.whl size=12620 sha256=0452bdac991364f65e6e5e7213fc9b5e31e631ef6909c3144a98e8e71a94e743
  Stored in directory: /root/.cache/pip/wheels/48/b4/d5/fb5b7f1d040f2ff20687e3bad6867d63155dbde5a7c10f4293
Successfully built gpustat
     |████████████████████████████████| 378kB 6.4MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5
Requirement already up-to-date: keras_preprocessing in /usr/local/lib/python3.6/dist-packages (1.1.0)
5c400d90da8c             Sun Dec 15 21:00:30 2019  418.67
[0] Tesla P100-PCIE-16GB | 43'C,   0 % |     0 / 16280 MB |


In [2]:
import numpy as np
import tensorflow as tf
import keras
import segmentation_models as sm
from keras.preprocessing.image import ImageDataGenerator
from google.colab import drive

drive.mount('/content/gdrive')

Using TensorFlow backend.


Segmentation Models: using `keras` framework.
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
data_dir = '/content/gdrive/My Drive/data_source/food_segmentation/unet/training_data/train/'
label_dir = '/content/gdrive/My Drive/data_source/food_segmentation/unet/training_label/train/'

In [5]:
BATCH_SIZE = 4
GENERATE_SEED = np.random.randint(0, 1024)
VALIDATION_SPLIT = 0.2

maskgen_args = dict(
    rotation_range=180,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=VALIDATION_SPLIT
)

imagegen_args = dict(
    samplewise_center=True,
    samplewise_std_normalization=True,
    **maskgen_args
)

image_datagen = ImageDataGenerator(**imagegen_args)
mask_datagen = ImageDataGenerator(**maskgen_args)

training_data_generator = zip(
    image_datagen.flow_from_directory(
        data_dir,
        target_size=(512, 512),
        color_mode='rgb',
        batch_size=BATCH_SIZE,
        class_mode=None,
        save_format='jpeg',
        seed=GENERATE_SEED,
        subset='training'
    ),
    mask_datagen.flow_from_directory(
        label_dir,
        target_size=(512, 512),
        color_mode='grayscale',
        batch_size=BATCH_SIZE,
        class_mode=None,
        save_format='png',
        seed=GENERATE_SEED,
        subset='training'
    )
)

validation_data_generator = zip(
    image_datagen.flow_from_directory(
        data_dir,
        target_size=(512, 512),
        color_mode='rgb',
        batch_size=BATCH_SIZE,
        class_mode=None,
        save_format='jpeg',
        seed=GENERATE_SEED,
        subset='validation'
    ),
    mask_datagen.flow_from_directory(
        label_dir,
        target_size=(512, 512),
        color_mode='grayscale',
        batch_size=BATCH_SIZE,
        class_mode=None,
        save_format='png',
        seed=GENERATE_SEED,
        subset='validation'
    )
)

Found 9672 images belonging to 1 classes.
Found 9672 images belonging to 1 classes.
Found 2418 images belonging to 1 classes.
Found 2418 images belonging to 1 classes.


In [6]:
unet_model = sm.Unet('resnet34', classes=1, activation='sigmoid')
unet_model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

85524480/85521592 [==============================] - 4s 0us/step
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
unet_model.fit_generator(
    training_data_generator, 
    validation_data=validation_data_generator,
    epochs=6,
    steps_per_epoch=2048,
    validation_steps=512,
    validation_freq=1,
    callbacks=[keras.callbacks.ModelCheckpoint("/content/gdrive/My Drive/data_source/food_segmentation/sm_unet/model_checkpoint_{epoch:02d}.hdf5")]
)